In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from ipywidgets import interact

# from conc_obj import EEGData
from eegdata_multi import EEGData
from plt import plot_psd, plot_montage
from ica import plot_ica_comp

# MNE imports
import mne
from mne.io.edf import read_raw_edf
from mne.datasets import eegbci

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

import json


***Macros***

>General use macros, importing JSON files to use as the configuration files

In [2]:
script_path = Path().resolve()
folder = (script_path / "../").resolve()

JSON_MAIN_PATH = script_path / "config_main.json"
JSON_CSP_PATH = script_path / "config_csp.json"
EVENTS_PATH = script_path / "events.json"

with open(JSON_MAIN_PATH, "r") as f:
    config_main = json.load(f)

with open(JSON_CSP_PATH, "r") as f:
    config_csp = json.load(f)

VERBOSE = config_main['verbose'].lower() == 'true'

L_FREQ = config_main['l_freq']
H_FREQ = config_main['h_freq']

N_SUBJECTS = config_main["n_subjects"]
N_COMPONENTS = config_main["n_components"]

"""
T0 corresponds to rest
T1 corresponds to onset of motion (real or imagined) of
the left fist (in runs 3, 4, 7, 8, 11, and 12)
both fists (in runs 5, 6, 9, 10, 13, and 14)
T2 corresponds to onset of motion (real or imagined) of
the right fist (in runs 3, 4, 7, 8, 11, and 12)
both feet (in runs 5, 6, 9, 10, 13, and 14)
"""


'\nT0 corresponds to rest\nT1 corresponds to onset of motion (real or imagined) of\nthe left fist (in runs 3, 4, 7, 8, 11, and 12)\nboth fists (in runs 5, 6, 9, 10, 13, and 14)\nT2 corresponds to onset of motion (real or imagined) of\nthe right fist (in runs 3, 4, 7, 8, 11, and 12)\nboth feet (in runs 5, 6, 9, 10, 13, and 14)\n'

***Initialization of EEG object***

>***(If the files are not locally stored, it will download them to the user system automatically)***

>***Use of functions like .filter_data() also is obligatory if there is no data stored locally***

In [3]:
eeg_obj = EEGData(config_main, config_csp, folder, verbose=VERBOSE)
# eeg_obj.save_type_data(type="events", folder_path=folder, verbose=VERBOSE)

#* Filters data and plots PSD to see differences
# eeg_obj.filter_data()
# eeg_obj.plot_psd_ba_filt(verbose=VERBOSE)

# eeg_obj.plot_psd(verbose=VERBOSE)

Loaded data:
<Raw | sample_mmi_h_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded> <Raw | sample_mmi_hf_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded>
<Raw | sample_mmi_h_filt_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded> <Raw | sample_mmi_hf_filt_raw.fif, 64 x 2333120 (14582.0 s), ~1.11 GiB, data loaded>


**Basic information and montage plotting in 2D & 3D**
> ***The channel names can also be printed***

In [4]:

#* Plots different montages in 2D & 3D
# data = eeg_obj.get_raw_h()
# ch_names = data.info["ch_names"] 
# plot_montage(eeg_obj.montage, ch_names)

**ICA(Independent Component Analysys)**
> ***The number of components that ICA will try to sort out can be changed, it is advised to use values in the range [15-45]***

> ***Ocular artifacts are also removed, since they don't contribute to the muscular movement on this evaluation***

> ***The components can also be plotted and ocular artifacts, EOG, will be clearly visible***

In [5]:
ev_list = config_csp["ev_mlist_three"]

data_h, data_hf = eeg_obj.get_filt()
events_h, events_hf = eeg_obj.get_events()

epochs, freq_bands = eeg_obj.crt_epochs(data_h, events_h, ev_list, "hands", verbose=VERBOSE)

features, labels = eeg_obj.simple_csp(epochs, freq_bands, verbose=VERBOSE)

#* Computes ICA components
# eeg_obj.decomp_ica(n_components=N_COMPONENTS, plt_show=True, verbose=VERBOSE)
features = features.reshape(features.shape[0], -1)
from sklearn.decomposition import FastICA
features = FastICA(n_components=N_COMPONENTS).fit_transform(features)

#* Plot components of ICA
# plot_ica_comp(folder / config["path_ica_h"])

#* Trains and evaluates model

#* Normalizes data
features = eeg_obj.normalize_data(features)
   
eeg_obj.train_model(features, labels)

Before :  [[2, 4]]
After :  {'do/right_hand': 2, 'imagine/right_hand': 4}
Using data from preloaded Raw for 872 events and 433 original time points ...
0 bad epochs dropped
(872, 64, 433)
(872,)
Computing rank from data with rank=None
    Using tolerance 0.00085 (2.2e-16 eps * 64 dim * 5.9e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.
(872, 64, 433)
(872,)
Computing rank from data with rank=None
    Using tolerance 0.00085 (2.2e-16 eps * 64 dim * 5.9e+10  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=2 covariance using EMPIRICAL
Done.
Estimating class=4 covariance using EMPIRICAL
Done.


/Users/Shared/42/ML/.venv/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:595: UserWarning: n_components is too large: it will be set to 16
  warnings.warn(
/Users/Shared/42/ML/.venv/lib/python3.12/site-packages/sklearn/decomposition/_fastica.py:128: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Train Accuracy LDA: 0.6413199426111909
Train Accuracy SVM: 1.0
Train Accuracy RF: 1.0
LDA Accuracy: 0.56
SVM Accuracy: 0.8914285714285715
Random Forest Accuracy: 0.9314285714285714
Confusion Matrix LDA:
 [[39 48]
 [29 59]]
Confusion Matrix SVM:
 [[84  3]
 [16 72]]
Confusion Matrix RF:
 [[81  6]
 [ 6 82]]


In [6]:
#* Saves filtered and concatenated data for faster loading
# eeg_obj.save_type_data(type="raw", folder_path=folder)
# eeg_obj.save_type_data(type="filtered", folder_path=folder)
eeg_obj.save_type_data(type="epochs", folder_path=folder)